# Entrega 2 Limpeza y preparación del dataset

\

In [ ]:
#descomprimir archivos
!ls
!unzip udea-ai4eng-20242.zip

sample_data  udea-ai4eng-20242.zip
Archive:  udea-ai4eng-20242.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


## Entrega final modleo 1

# solucion 2


In [ ]:
!pip install xgboost

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

class SaberProXGBoostPipeline:
    def __init__(self):
        self.pipeline = None
        self.label_encoder = LabelEncoder()

    def create_preprocessing_pipeline(self, df):
        """
        Crea un pipeline de preprocesamiento optimizado para XGBoost
        """
        # Mapeos para variables categóricas
        self.estrato_map = {
            "Estrato 1": 1, "Estrato 2": 2, "Estrato 3": 3,
            "Estrato 4": 4, "Estrato 5": 5, "Estrato 6": 6
        }

        self.horas_trabajo_map = {
            '0': 0, 'Menos de 10 horas': 5,
            'Entre 11 y 20 horas': 15, 'Entre 21 y 30 horas': 25,
            'Más de 30 horas': 30
        }

        self.valor_matricula_map = {
            'Entre 1 millón y menos de 2.5 millones': 1.75,
            'Entre 2.5 millones y menos de 4 millones': 3.25,
            'Menos de 500 mil': 0.25,
            'Entre 500 mil y menos de 1 millón': 0.75,
            'Entre 4 millones y menos de 5.5 millones': 4.75,
            'Más de 7 millones': 7.75,
            'Entre 5.5 millones y menos de 7 millones': 6.25,
            'No pagó matrícula': 0
        }

        # Características numéricas y categóricas
        numeric_features = ['FAMI_ESTRATOVIVIENDA', 'ESTU_HORASSEMANATRABAJA',
                          'ESTU_VALORMATRICULAUNIVERSIDAD', 'FAMI_TIENEINTERNET',
                          'ESTU_PAGOMATRICULAPROPIO']

        categorical_features = ['ESTU_PRGM_DEPARTAMENTO']

        # Transformadores
        numeric_transformer = Pipeline(steps=[
            ('scaler', StandardScaler())
        ])

        categorical_transformer = Pipeline(steps=[
            ('onehot', OneHotEncoder(drop='first', sparse_output=False))
        ])

        # Preprocessor
        preprocessor = ColumnTransformer(
            transformers=[
                ('num', numeric_transformer, numeric_features),
                ('cat', categorical_transformer, categorical_features)
            ])

        # XGBoost con configuración inicial optimizada
        xgb_classifier = XGBClassifier(
            n_estimators=500,
            learning_rate=0.1,
            max_depth=6,
            min_child_weight=1,
            gamma=0,
            subsample=0.8,
            colsample_bytree=0.8,
            objective='multi:softprob',
            random_state=42
        )

        # Pipeline completo
        self.pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('classifier', xgb_classifier)
        ])

        return self.pipeline

    def prepare_data(self, df):
        """
        Prepara los datos aplicando las transformaciones necesarias
        """
        df_processed = df.copy()

        # Eliminar columnas no necesarias
        columns_to_drop = ['ID', 'PERIODO', 'FAMI_EDUCACIONPADRE',
                          'FAMI_EDUCACIONMADRE', 'ESTU_PRGM_ACADEMICO']
        df_processed = df_processed.drop(columns_to_drop, axis=1, errors='ignore')

        # Aplicar mapeos con manejo de valores faltantes
        df_processed['FAMI_ESTRATOVIVIENDA'] = df_processed['FAMI_ESTRATOVIVIENDA'].fillna('Estrato 3').map(self.estrato_map)
        df_processed['ESTU_HORASSEMANATRABAJA'] = df_processed['ESTU_HORASSEMANATRABAJA'].fillna('Más de 30 horas').map(self.horas_trabajo_map)
        df_processed['ESTU_VALORMATRICULAUNIVERSIDAD'] = df_processed['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna('Entre 1 millón y menos de 2.5 millones').map(self.valor_matricula_map)
        df_processed['FAMI_TIENEINTERNET'] = df_processed['FAMI_TIENEINTERNET'].fillna('Si').map({'Si': 1, 'No': 0})
        df_processed['ESTU_PAGOMATRICULAPROPIO'] = df_processed['ESTU_PAGOMATRICULAPROPIO'].fillna('No').map({'Si': 1, 'No': 0})

        return df_processed

    def train_model(self, X_train, y_train):
        """
        Entrena el modelo usando GridSearchCV con parámetros optimizados para XGBoost
        """
        param_grid = {
            'classifier__max_depth': [3, 5, 7],
            'classifier__min_child_weight': [1, 3, 5],
            'classifier__gamma': [0.0, 0.1, 0.2],
            'classifier__learning_rate': [0.01, 0.1],
            'classifier__n_estimators': [200, 500],
            'classifier__subsample': [0.8, 0.9],
            'classifier__colsample_bytree': [0.8, 0.9]
        }

        # Validación cruzada estratificada
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

        grid_search = GridSearchCV(
            self.pipeline,
            param_grid,
            cv=cv,
            scoring='accuracy',
            n_jobs=-1,
            verbose=2
        )

        grid_search.fit(X_train, y_train)

        print("Mejores parámetros encontrados:")
        print(grid_search.best_params_)

        self.pipeline = grid_search.best_estimator_
        return grid_search.best_score_

    def evaluate_model(self, X_test, y_test):
        """
        Evalúa el modelo y genera visualizaciones detalladas
        """
        y_pred = self.pipeline.predict(X_test)
        y_prob = self.pipeline.predict_proba(X_test)

        # Reporte de clasificación
        print("\nReporte de Clasificación:")
        print(classification_report(y_test, y_pred))

        # Matriz de confusión
        plt.figure(figsize=(10, 8))
        cm = confusion_matrix(y_test, y_pred)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.title('Matriz de Confusión')
        plt.ylabel('Valor Real')
        plt.xlabel('Predicción')
        plt.show()

        # Importancia de características
        feature_importance = pd.DataFrame(
            self.pipeline.named_steps['classifier'].feature_importances_,
            index=self.pipeline.named_steps['preprocessor'].get_feature_names_out(),
            columns=['importance']
        ).sort_values('importance', ascending=False)

        plt.figure(figsize=(12, 6))
        feature_importance.head(10).plot(kind='bar')
        plt.title('Top 10 Características más Importantes')
        plt.tight_layout()
        plt.show()

        return feature_importance

def main():
    # Cargar datos
    train_df = pd.read_csv("train.csv")
    test_df = pd.read_csv("test.csv")

    # Crear instancia del pipeline
    pipeline = SaberProXGBoostPipeline()

    # Separar features y target
    X_train = train_df.drop(['RENDIMIENTO_GLOBAL'], axis=1)
    y_train = train_df['RENDIMIENTO_GLOBAL']

    # Codificar target
    y_train_encoded = pipeline.label_encoder.fit_transform(y_train)

    # Crear y configurar pipeline
    pipeline.create_preprocessing_pipeline(train_df)

    # Preparar datos
    X_train_processed = pipeline.prepare_data(X_train)

    # Entrenar modelo
    print("Entrenando modelo XGBoost...")
    best_score = pipeline.train_model(X_train_processed, y_train_encoded)
    print(f"\nMejor score en validación cruzada: {best_score:.3f}")

    # Preparar y guardar predicciones
    test_processed = pipeline.prepare_data(test_df)
    predictions = pipeline.pipeline.predict(test_processed)
    predictions = pipeline.label_encoder.inverse_transform(predictions)

    submission = pd.DataFrame({
        'ID': test_df['ID'],
        'RENDIMIENTO_GLOBAL': predictions
    })

    submission.to_csv('submission_xgboost.csv', index=False)
    print("\nSubmission guardada como 'submission_xgboost.csv'")

    return pipeline, submission

if __name__ == "__main__":
    pipeline, submission = main()

Entrenando modelo XGBoost...
Fitting 5 folds for each of 432 candidates, totalling 2160 fits
